# Equivalent Circuit Parameter Identification
## Estimating resistance and capacitance values from HPPC data

This notebook demonstrates how to identify stationary parameters for a Thevenin model. The Thevenin model is an empirical circuit model capable of capturing the electrical response of a battery. This model can be extended with a thermal submodel, as well as additional parallel resistor-capacitor (RC) branches.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by upgrading some dependencies and importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
%pip install --upgrade pip pandas -q

import numpy as np
import pandas as pd
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


## Importing parameters

First let's create the model with two RC elements.

In [ ]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})

Parameters can be defined by importing a JSON representation, such as the one in the PyBOP [examples](https://github.com/pybop-team/PyBOP/blob/develop/examples/parameters/initial_ecm_parameters.json). To import via JSON, either download the example file, or create your own and update the path below to reference the corresponding file.

In [ ]:
# import json

# json_path = "../../parameters/initial_ecm_parameters.json"
# with open(json_path) as f:
#     params_dict = json.load(f)

# parameter_values = pybamm.ParameterValues(params_dict)
# parameter_values.update(
#     {"Open-circuit voltage [V]": model.default_parameter_values["Open-circuit voltage [V]"]},
#     check_already_exists=False,
# )

Alternatively, define the initial parameter set with a dictionary. Ensure you have definitions for all R's, C's, and initial overpotentials for any additional RC elements.

In this example, we use the default parameter value for the "Open-circuit voltage [V]" as provided by the original PyBaMM class. To update this, provide a function definition that matches this [function](https://github.com/pybamm-team/PyBaMM/blob/1943aa5ab2895b5378220595923dbae3d66b13c9/pybamm/input/parameters/ecm/example_set.py#L17).

In [ ]:
parameter_values = model.default_parameter_values
parameter_values.update(
    {
        "Initial SoC": 0.5,
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Current function [A]": 5,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "Element-1 initial overpotential [V]": 0,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
    }
)
# Optional arguments - only needed for two RC pairs
parameter_values.update(
    {
        "Element-2 initial overpotential [V]": 0,
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
    },
    check_already_exists=False,
)

## Importing Data

We will use experimental data for a single HPPC pulse from an open dataset [1]. This is imported and used to construct the `pybop.Dataset` class.

[1] Kollmeyer, Phillip; Skells, Michael (2020), “Samsung INR21700 30T 3Ah Li-ion Battery Data”, Mendeley Data, V1, doi: 10.17632/9xyvy2njj3.1

In [ ]:
file_loc = r"../../data/Samsung_INR21700/sample_hppc_pulse.xlsx"
df = pd.read_excel(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(),
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)
parameter_values.set_initial_state(f"{df['Voltage'].to_numpy()[0]} V");

## Identifying the parameters

Now that the initial parameter set is constructed, we can start the PyBOP fitting process. In this example, we've construct a two-branch Thevenin model, so we will select all five resistance and capacitance parameters for identification. This isn't recommended for real-life application as it is difficult to guarantee identifiablity with this large a parameter space. The initial guess for each resistance parameter is generated from a random sample of the prior distributions. These are influenced by the `r_guess` parameter below.

In [ ]:
r_guess = 0.005
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[100, 10000],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[100, 10000],
    ),
]

Now we construct the problem class. We add the dataset, model and parameter values. We choose sum squared error as the cost for the optimisation problem.

In [ ]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model, parameter_values=parameter_values)
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)
problem = builder.build()

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the XNES method as it provides global optimisation capability. For the sake of reducing the runtime of this example, we limit the maximum iterations; however, feel free to update this value. Due to the scale differences in the parameters, we update the optimiser step-size (`sigma`) to be parameter specific, which helps ensure the optimiser explores the complete parameter space.

In [ ]:
options = pybop.PintsOptions(
    sigma=[1e-3, 1e-3, 1e-3, 20, 20],
    max_unchanged_iterations=30,
    max_iterations=100,
)
optim = pybop.XNES(problem, options=options)
results = optim.run()

## Plotting and Visualisation

Next, we use PyBOP's plotting utilities to visualise the results of the optimisation. This provides us with a visual confirmation of the optimiser's converged parameter values in the time-domain output.

In [ ]:
sim = pybamm.Simulation(model, parameter_values=results.parameter_values)
t_eval = dataset["Time [s]"]
sol = sim.solve(t_eval=t_eval)
pybop.plot.trajectories(
    x=t_eval,
    y=[dataset["Voltage [V]"], sol["Voltage [V]"](t_eval)],
    trace_names=["Dataset", "Simulation"],
    xaxis_title="Time / s",
    yaxis_title="Voltage / V",
);

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning:

The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.



### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

### Conclusion

This notebook illustrates how to perform parameter estimation in PyBOP, providing insights into the optimisation process through various visualisations.